# Group Assignment - Natural Language Processing
##### Dennis Depoi, Anne Szyprons, Leonie Schulz, Melanie Ullrich

## Pipeline

1. Einlesen der Daten
    - alle xml-Dateien zusammenführen
    - Beschränkung auf eine Wahlperiode 
2. Datentransformation
    - Datenbereinigung (Entfernen der überflüssigen Zeichen)
    - Bereinigung der Spalte "TEXT"
    - 3 Ansätze:
        - Personenerkennung mit NER mit Spacy und nltk hat nicht funktioniert 
        - Extrahierung der Teilnehmer der Sitzungen anhand des Namensverzeichnis
        - Verwendung der Stammdatendatei (xml)
        - manuelle Liste der Abgeordneten in der Wahlperiode
    - Erstellung eines Subdataframes (Name des Vortragenden, Unterbrechungen, etc (Präsi))
3. Datenverarbeitung
    - Case Conversion
    - Tokenisierung
    - Entfernen von Stopwörtern
    - Lemmatisierung
    
    
4. Anwendung von Methoden: 
- Anwendung von Statistik: Häufigkeitsberechnung (Wie oft wer unterbrochen?)
- bag of words - tf-idf Maß - was sind häufige Wörter?
- Clustering? Einteilung in Klassen nach Wörter (neutral, Zustimmung, Ablehung)


Erstellung TF-IDF-Matrix (sklearns TfidfVectorizer)
Clustering? Topic Modelling?



## To Do
- Bereinigung der Textzeilen
- if-Schleife erweitern für mega Daten (Teilnehmer)

# 1. Einlesen der Daten

In [1]:
#settings, imports & downloads
import pandas as pd
import os
from bs4 import BeautifulSoup
import re
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
import spacy

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')
#! python -m spacy download de_core_news_md

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/dennisdepoi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/dennisdepoi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/dennisdepoi/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/dennisdepoi/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
#settings
pd.options.display.max_colwidth = 200

In [4]:
# read xml files
folder_path = "data"
files_in_folder = os.listdir(folder_path)

result = []
for file in files_in_folder:
    soup = BeautifulSoup(open(folder_path + "/" + file, "r", encoding="utf-8"), "xml")
    d = {}
    for tag in soup.find_all(recursive=True):
        d[tag.name] = tag.get_text(strip=True)
        result.append(d)
        
#write in Dataframe
df = pd.DataFrame(result)
df = df.drop_duplicates()
df = df.drop(columns = "DOKUMENT") #delete redundant column
#df.to_csv("data_protokoll.csv") #write to csv file
df = df[50:60]
df

,WAHLPERIODE,DOKUMENTART,NR,DATUM,TITEL,TEXT
350,17,PLENARPROTOKOLL,17/15,20.01.2010,Plenarprotokoll vom 20.01.2010,"Plenarprotokoll 17/15\nDeutscher Bundestag\nStenografischer Bericht\n\n15. Sitzung\n\nBerlin, Mittwoch, den 20. Januar 2010\n\nI n h a l t :\nTagesordnungspunkt 2 (Fortsetzung):\n\nErste Beratung ..."
357,17,PLENARPROTOKOLL,17/29,16.03.2010,Plenarprotokoll vom 16.03.2010,"Plenarprotokoll 17/29\nDeutscher Bundestag\nStenografischer Bericht\n\n29. Sitzung\n\nBerlin, Dienstag, den 16. März 2010\n\nI n h a l t :\nGlückwünsche zum Geburtstag des Abgeord-\nneten Dr. Herb..."
364,17,PLENARPROTOKOLL,17/203,07.11.2012,Plenarprotokoll vom 07.11.2012,"Plenarprotokoll 17/203\n\nDeutscher Bundestag\nStenografischer Bericht\n\n203. Sitzung\n\nBerlin, Mittwoch, den 7. November 2012\n\nI n h a l t :\n\nTagesordnungspunkt 1:\nBefragung der Bundesregi..."
371,17,PLENARPROTOKOLL,17/217,17.01.2013,Plenarprotokoll vom 17.01.2013,"Plenarprotokoll 17/217\n\nDeutscher Bundestag\nStenografischer Bericht\n\n217. Sitzung\n\nBerlin, Donnerstag, den 17. Januar 2013\n\nI n h a l t :\n\nGlückwünsche zum Geburtstag der Abgeord-\nnete..."
378,17,PLENARPROTOKOLL,17/216,16.01.2013,Plenarprotokoll vom 16.01.2013,"Plenarprotokoll 17/216Inhaltsverzeichnis\nDeutscher Bundestag\nStenografischer Bericht\n\n216. Sitzung\n\nBerlin, Mittwoch, den 16. Januar 2013\n\nI n h a l t :\nAbsetzung des Tagesordnungspunktes..."
385,17,PLENARPROTOKOLL,17/202,26.10.2012,Plenarprotokoll vom 26.10.2012,"Plenarprotokoll 17/202\n\nDeutscher Bundestag\nStenografischer Bericht\n\n202. Sitzung\n\nBerlin, Freitag, den 26. Oktober 2012\n\nI n h a l t :\n\nTagesordnungspunkt 43:\n\nErste Beratung des von..."
392,17,PLENARPROTOKOLL,17/28,05.03.2010,Plenarprotokoll vom 05.03.2010,"Plenarprotokoll 17/28\nDeutscher Bundestag\nStenografischer Bericht\n\n28. Sitzung\n\nBerlin, Freitag, den 5. März 2010\n\nI n h a l t :\nGlückwünsche zum Geburtstag der Abgeord-\nneten Ulla Lötze..."
399,17,PLENARPROTOKOLL,17/14,19.01.2010,Plenarprotokoll vom 19.01.2010,"Plenarprotokoll 17/14\nDeutscher Bundestag\nStenografischer Bericht\n\n14. Sitzung\n\nBerlin, Dienstag, den 19. Januar 2010\n\nI n h a l t :\nGlückwünsche zum Geburtstag der Abgeord-\nneten Franz ..."
406,17,PLENARPROTOKOLL,17/148,14.12.2011,Plenarprotokoll vom 14.12.2011,"Plenarprotokoll 17/148\nDeutscher Bundestag\nStenografischer Bericht\n\n148. Sitzung\n\nBerlin, Mittwoch, den 14. Dezember 2011\n\nI n h a l t :\nErweiterung und Abwicklung der Tagesord-\nnung . ..."
413,17,PLENARPROTOKOLL,17/160,27.02.2012,Plenarprotokoll vom 27.02.2012,"Plenarprotokoll 17/160Inhaltsverzeichnis\nDeutscher Bundestag\nStenografischer Bericht\n\n160. Sitzung\n\nBerlin, Montag, den 27. Februar 2012\n\nI n h a l t :\nTagesordnungspunkt 1:\n\na) Abgabe ..."


In [ ]:
'''
import xml.etree.ElementTree as et 

xtree = et.parse("MDB_STAMMDATEN.xml")
xroot = xtree.getroot()

df_cols = []
rows = []

for child in xroot:
    df_cols_ = df_cols.append(child.tag)
    rows_ = rows.append(child.text)

out_df = pd.DataFrame(rows, columns = df_cols)'''

In [ ]:
'''# read xml files
result = []
soup = BeautifulSoup(open("MDB_STAMMDATEN.xml", "r", encoding="utf-8"), "xml")
d = {}
for tag in soup.find_all(recursive=True):
    d[tag.name] = tag.get_text(strip=True)
    result.append(d)
        
#write in Dataframe
stammdaten = pd.DataFrame(result)'''

In [ ]:
'''stammdaten = stammdaten[:10]
stammdaten = stammdaten.drop_duplicates()
stammdaten = stammdaten["WAHLPERIODE" == "17"]'''

In [5]:
#read csv file of the names
gender_df = pd.read_csv("DIS25_liste_gender.csv",delimiter=";")

In [6]:
names = gender_df["Vorname_Nachname"].tolist()
partei = gender_df["Fraktion"].tolist()

# 2. Datentransformation

In [ ]:
'''
#extract the participants in columns "TEXT"
pattern_names = "Namensverzeichnis"
participants = []
for row in df["TEXT"]:
    if str(row).find(pattern_names):
        rowstr = str(row).split(pattern_names)
        participants_ = participants.append(rowstr[1].split("\n"))

#print(participants)
df["TEILNEHMER"] = participants '''

In [7]:
#split column "TEXT" and add only the important text into new column and the participants
conversation = []
pattern_start = "Beginn:"
pattern_end = "Schluss:"

for row in df["TEXT"]:
    rowstr = str(row).split(pattern_start)
    conversation.append(rowstr[1])

df["TEXT_SPLIT"] = conversation

In [8]:
df

,WAHLPERIODE,DOKUMENTART,NR,DATUM,TITEL,TEXT,TEXT_SPLIT
350,17,PLENARPROTOKOLL,17/15,20.01.2010,Plenarprotokoll vom 20.01.2010,"Plenarprotokoll 17/15\nDeutscher Bundestag\nStenografischer Bericht\n\n15. Sitzung\n\nBerlin, Mittwoch, den 20. Januar 2010\n\nI n h a l t :\nTagesordnungspunkt 2 (Fortsetzung):\n\nErste Beratung ...","9.01 Uhr\nPräsident Dr. Norbert Lammert:\nDie Sitzung ist eröffnet.\n\nGuten Morgen, liebe Kolleginnen und Kollegen! Wir\nsetzen unsere Haushaltsberatungen – Tagesordnungs-\npunkt 2 – fort:\n\nEr..."
357,17,PLENARPROTOKOLL,17/29,16.03.2010,Plenarprotokoll vom 16.03.2010,"Plenarprotokoll 17/29\nDeutscher Bundestag\nStenografischer Bericht\n\n29. Sitzung\n\nBerlin, Dienstag, den 16. März 2010\n\nI n h a l t :\nGlückwünsche zum Geburtstag des Abgeord-\nneten Dr. Herb...","10.00 Uhr\nPräsident Dr. Norbert Lammert:\nDie Sitzung ist eröffnet. Nehmen Sie bitte Platz.\n\nGuten Morgen, liebe Kolleginnen und Kollegen! Ich\nbegrüße Sie alle herzlich zu den Haushaltsberatu..."
364,17,PLENARPROTOKOLL,17/203,07.11.2012,Plenarprotokoll vom 07.11.2012,"Plenarprotokoll 17/203\n\nDeutscher Bundestag\nStenografischer Bericht\n\n203. Sitzung\n\nBerlin, Mittwoch, den 7. November 2012\n\nI n h a l t :\n\nTagesordnungspunkt 1:\nBefragung der Bundesregi...",13.00 Uhr\n\nVizepräsident Eduard Oswald:\nIch grüße Sie sehr herzlich. Schönen Nachmittag! Die\n\nSitzung ist eröffnet.\n\nIch rufe den Tagesordnungspunkt 1 auf:\n\nBefragung der Bundesregierung...
371,17,PLENARPROTOKOLL,17/217,17.01.2013,Plenarprotokoll vom 17.01.2013,"Plenarprotokoll 17/217\n\nDeutscher Bundestag\nStenografischer Bericht\n\n217. Sitzung\n\nBerlin, Donnerstag, den 17. Januar 2013\n\nI n h a l t :\n\nGlückwünsche zum Geburtstag der Abgeord-\nnete...","9.00 Uhr\n\nPräsident Dr. Norbert Lammert:\nNehmen Sie bitte Platz. Die Sitzung ist eröffnet.\n\nGuten Morgen, liebe Kolleginnen und Kollegen! Ich\nbegrüße Sie alle herzlich und fange dabei mit d..."
378,17,PLENARPROTOKOLL,17/216,16.01.2013,Plenarprotokoll vom 16.01.2013,"Plenarprotokoll 17/216Inhaltsverzeichnis\nDeutscher Bundestag\nStenografischer Bericht\n\n216. Sitzung\n\nBerlin, Mittwoch, den 16. Januar 2013\n\nI n h a l t :\nAbsetzung des Tagesordnungspunktes...","11.00 Uhr\nPräsident Dr. Norbert Lammert:\nDie Sitzung ist eröffnet. Nehmen Sie bitte einen Au-\n\ngenblick Platz.\n\nLiebe Kolleginnen und Kollegen, ich begrüße Sie alle\nherzlich zur ersten Ple..."
385,17,PLENARPROTOKOLL,17/202,26.10.2012,Plenarprotokoll vom 26.10.2012,"Plenarprotokoll 17/202\n\nDeutscher Bundestag\nStenografischer Bericht\n\n202. Sitzung\n\nBerlin, Freitag, den 26. Oktober 2012\n\nI n h a l t :\n\nTagesordnungspunkt 43:\n\nErste Beratung des von...","9.00 Uhr\n\nVizepräsidentin Katrin Göring-Eckardt:\nGuten Morgen, liebe Kolleginnen und Kollegen! Die\n\nSitzung ist eröffnet.\n\nIch rufe den Tagesordnungspunkt 43 auf:\n\nErste Beratung des von..."
392,17,PLENARPROTOKOLL,17/28,05.03.2010,Plenarprotokoll vom 05.03.2010,"Plenarprotokoll 17/28\nDeutscher Bundestag\nStenografischer Bericht\n\n28. Sitzung\n\nBerlin, Freitag, den 5. März 2010\n\nI n h a l t :\nGlückwünsche zum Geburtstag der Abgeord-\nneten Ulla Lötze...","9.00 Uhr\nPräsident Dr. Norbert Lammert:\nDie Sitzung ist eröffnet. Guten Morgen, liebe Kolle-\n\nginnen und Kollegen! Ich begrüße Sie alle herzlich.\n\nDie Kollegin Ulla Lötzer feiert heute ihre..."
399,17,PLENARPROTOKOLL,17/14,19.01.2010,Plenarprotokoll vom 19.01.2010,"Plenarprotokoll 17/14\nDeutscher Bundestag\nStenografischer Bericht\n\n14. Sitzung\n\nBerlin, Dienstag, den 19. Januar 2010\n\nI n h a l t :\nGlückwünsche zum Geburtstag der Abgeord-\nneten Franz ...",10.00 Uhr\nPräsident Dr. Norbert Lammert:\nDie Sitzung ist eröffnet.\n\nLiebe Kolleginnen und Kollegen! Ich begrüße Sie alle\nherzlich zur ersten Plenarsitzung des Deutschen Bundes-\ntages im neu...
406,17,PLENARPROTOKOLL,17/148,14.12.2011,Plenarprotokoll vom 14.12.2011,"Plenarprotokoll 17/148\nDeutscher Bundestag\nStenografischer Bericht\n\n148. Si

In [ ]:
'''#split column "TEXT" and add only the important text into new column and the participants
conversation = []
participants = []
pattern_start = "Beginn:"
pattern_end = "Schluss:"
pattern_names = "Namensverzeichnis"

for row in df["TEXT"]:
    rowstr_start = str(row).split(pattern_start)
    rowstr_start1 = rowstr_start[1]
    rowstr_names = rowstr_start1.split(pattern_names)
    conversation.append(rowstr_names[0])
    participants.append(rowstr_names[1])

df["TEXT_SPLIT"] = conversation
df["TEILNEHMER"] = participants'''

In [9]:
#clean data
df["TEXT_SPLIT"] = df.apply(lambda row: re.sub("(\n)|(\r)|(\s+)", " ", row["TEXT_SPLIT"]), axis=1) #delete returns in data
df["TEXT_SPLIT"] = df.apply(lambda row: re.sub("-\s", "", row["TEXT_SPLIT"]), axis=1) #delete separate words with hyphen

In [10]:
df

,WAHLPERIODE,DOKUMENTART,NR,DATUM,TITEL,TEXT,TEXT_SPLIT
350,17,PLENARPROTOKOLL,17/15,20.01.2010,Plenarprotokoll vom 20.01.2010,"Plenarprotokoll 17/15\nDeutscher Bundestag\nStenografischer Bericht\n\n15. Sitzung\n\nBerlin, Mittwoch, den 20. Januar 2010\n\nI n h a l t :\nTagesordnungspunkt 2 (Fortsetzung):\n\nErste Beratung ...","9.01 Uhr Präsident Dr. Norbert Lammert: Die Sitzung ist eröffnet. Guten Morgen, liebe Kolleginnen und Kollegen! Wir setzen unsere Haushaltsberatungen – Tagesordnungspunkt 2 – fort: Erste Beratu..."
357,17,PLENARPROTOKOLL,17/29,16.03.2010,Plenarprotokoll vom 16.03.2010,"Plenarprotokoll 17/29\nDeutscher Bundestag\nStenografischer Bericht\n\n29. Sitzung\n\nBerlin, Dienstag, den 16. März 2010\n\nI n h a l t :\nGlückwünsche zum Geburtstag des Abgeord-\nneten Dr. Herb...","10.00 Uhr Präsident Dr. Norbert Lammert: Die Sitzung ist eröffnet. Nehmen Sie bitte Platz. Guten Morgen, liebe Kolleginnen und Kollegen! Ich begrüße Sie alle herzlich zu den Haushaltsberatungen ..."
364,17,PLENARPROTOKOLL,17/203,07.11.2012,Plenarprotokoll vom 07.11.2012,"Plenarprotokoll 17/203\n\nDeutscher Bundestag\nStenografischer Bericht\n\n203. Sitzung\n\nBerlin, Mittwoch, den 7. November 2012\n\nI n h a l t :\n\nTagesordnungspunkt 1:\nBefragung der Bundesregi...",13.00 Uhr Vizepräsident Eduard Oswald: Ich grüße Sie sehr herzlich. Schönen Nachmittag! Die Sitzung ist eröffnet. Ich rufe den Tagesordnungspunkt 1 auf: Befragung der Bundesregierung Die Bun...
371,17,PLENARPROTOKOLL,17/217,17.01.2013,Plenarprotokoll vom 17.01.2013,"Plenarprotokoll 17/217\n\nDeutscher Bundestag\nStenografischer Bericht\n\n217. Sitzung\n\nBerlin, Donnerstag, den 17. Januar 2013\n\nI n h a l t :\n\nGlückwünsche zum Geburtstag der Abgeord-\nnete...","9.00 Uhr Präsident Dr. Norbert Lammert: Nehmen Sie bitte Platz. Die Sitzung ist eröffnet. Guten Morgen, liebe Kolleginnen und Kollegen! Ich begrüße Sie alle herzlich und fange dabei mit den Sch..."
378,17,PLENARPROTOKOLL,17/216,16.01.2013,Plenarprotokoll vom 16.01.2013,"Plenarprotokoll 17/216Inhaltsverzeichnis\nDeutscher Bundestag\nStenografischer Bericht\n\n216. Sitzung\n\nBerlin, Mittwoch, den 16. Januar 2013\n\nI n h a l t :\nAbsetzung des Tagesordnungspunktes...","11.00 Uhr Präsident Dr. Norbert Lammert: Die Sitzung ist eröffnet. Nehmen Sie bitte einen Au genblick Platz. Liebe Kolleginnen und Kollegen, ich begrüße Sie alle herzlich zur ersten Plenarsitzun..."
385,17,PLENARPROTOKOLL,17/202,26.10.2012,Plenarprotokoll vom 26.10.2012,"Plenarprotokoll 17/202\n\nDeutscher Bundestag\nStenografischer Bericht\n\n202. Sitzung\n\nBerlin, Freitag, den 26. Oktober 2012\n\nI n h a l t :\n\nTagesordnungspunkt 43:\n\nErste Beratung des von...","9.00 Uhr Vizepräsidentin Katrin Göring-Eckardt: Guten Morgen, liebe Kolleginnen und Kollegen! Die Sitzung ist eröffnet. Ich rufe den Tagesordnungspunkt 43 auf: Erste Beratung des von der Bund..."
392,17,PLENARPROTOKOLL,17/28,05.03.2010,Plenarprotokoll vom 05.03.2010,"Plenarprotokoll 17/28\nDeutscher Bundestag\nStenografischer Bericht\n\n28. Sitzung\n\nBerlin, Freitag, den 5. März 2010\n\nI n h a l t :\nGlückwünsche zum Geburtstag der Abgeord-\nneten Ulla Lötze...","9.00 Uhr Präsident Dr. Norbert Lammert: Die Sitzung ist eröffnet. Guten Morgen, liebe Kolle ginnen und Kollegen! Ich begrüße Sie alle herzlich. Die Kollegin Ulla Lötzer feiert heute ihren 60. Ge..."
399,17,PLENARPROTOKOLL,17/14,19.01.2010,Plenarprotokoll vom 19.01.2010,"Plenarprotokoll 17/14\nDeutscher Bundestag\nStenografischer Bericht\n\n14. Sitzung\n\nBerlin, Dienstag, den 19. Januar 2010\n\nI n h a l t :\nGlückwünsche zum Geburtstag der Abgeord-\nneten Franz ...",10.00 Uhr Präsident Dr. Norbert Lammert: Die Sitzung ist eröffnet. Liebe Kolleginnen und Kollegen! Ich begrüße Sie alle herzlich zur ersten Plenarsitzung des Deutschen Bundestages im neuen Jahr....
406,17,PLENARPROTOKOLL,17/148,14.12.2011,Plenarprotokoll vom 14.12.2011,"Plenarprotokoll 17/148\nDeutscher Bundestag\nStenografischer Bericht\n\n148. Sitzung\n\nBerlin, 

## Neues SubDataframe

In [ ]:
'''row = "11.00 Uhr Alterspräsident Dr. Heinz Riesenhuber: Guten Morgen, meine sehr verehrten Damen und Her ren! Liebe Kolleginnen und Kollegen, ich begrüße Sie zur konstituierenden Sitzung des 17. Deutschen Bundestags.  Parlamentarischer Brauch ist es – das entspricht unserer Geschäftsordnung; ich kann die Paragrafen zitieren –, dass der Älteste die erste Sitzung des Bundestags eröffnet. Ich bin am Sonntag, dem 1. Dezember 1935, geboren. Wenn jemand von den Kollegen im Saal älter ist als ich, dann spreche er jetzt oder er schweige für immer.  (Heiterkeit und Beifall) "
x = "Heinz Riesenhuber"

start = row.index(str(x) + ":")
end = row.index(')',start+1)
substring = row[start-1:end+1]
    

print(f"Start: {start}, End: {end}")
print(substring)'''

In [11]:
# write the components of speaker, speech and interruption
Sub = []

for row in df["TEXT_SPLIT"]:
    for i in names:
        x = str(i) + ":"
        if x in row:
            start = str(row).index(x)
            end = row.index(')',start+1)
            substring = row[start-1:end+1]
            Sub.append(substring)

In [12]:
#split the components and write into new dataframe
data = pd.DataFrame()
Redebeitrag = []
Person = []
Unterbrechung = []

for x in Sub:
    string = re.split(':', str(x), 1)
    string1 = re.split('\\(|\\)', str(string[1]))
    Person.append(string[0])
    Redebeitrag.append(string1[0])
    Unterbrechung.append(string1[1])

data["Redeinhalt"] = Redebeitrag
data["Vortragende_Person"] = Person
data["Unterbrechung"] = Unterbrechung

In [13]:
#drop columns with no relevant interruptions
data.drop(data[data["Unterbrechung"] == "A"].index, inplace = True)

In [14]:
data = data.loc[~data['Unterbrechung'].isin(partei)]
data = data[~data['Unterbrechung'].str.lower().str.contains("beifall")]

In [26]:
data = data[~data["Unterbrechung"].isnull()] #funktioniert nicht!

In [27]:
data

,Redeinhalt,Vortragende_Person,Unterbrechung,TOKENS,POS,LEMMATA
1,"Die Sitzung ist eröffnet. Guten Morgen, liebe Kolleginnen und Kollegen! Wir setzen unsere Haushaltsberatungen – Tagesordnungspunkt 2 – fort: Erste Beratung des von der Bundesregierung eingebrac...",Norbert Lammert,Haushaltsgesetz 2010,"[haushaltsgesetz, 2010]","[(haushaltsgesetz, NN), (2010, CD)]","[haushaltsgesetz, 2010]"
13,Ich eröffne die unterbrochene Sitzung wieder und teile die Ergebnisse der drei namentlichen Abstimmungen mit. Zunächst gebe ich das von den Schriftführerinnen und Schriftführern ermittelte Erge...,Wolfgang Thierse,D,[d],"[(d, NN)]",[d]
14,"„Die Deutschen haben ein Recht auf Widerstand gegen die Überfremdung“, Ausdrücke auf Plakaten wie auf denen der CDU: „Asylmissbrauch beenden“, usw. waren damals auch die Stichworte für die Brandl...",Norbert Geis,Dr. Heinrich L. Kolb [FDP]: Gewagte These!,"[dr, heinrich, l, kolb, fdp, gewagte, these]","[(dr, NN), (heinrich, NN), (l, NN), (kolb, NN), (fdp, NN), (gewagte, NN), (these, DT)]","[dr, heinrich, l, kolb, fdp, gewagte, these]"
16,"Unterstützt die Bundesregierung finanziell die von Indo nesien bestätigte Lieferung von 130 gebrauchten Leopard-2Panzern, und sind diese Panzer – ebenso wie die für SaudiArabien vorgesehene Version",Rolf Mützenich,vergleiche Frankfurter Allgemeine Sonntagszeitung vom 12. August 2012,"[vergleiche, frankfurter, allgemeine, sonntagszeitung, 12, august, 2012]","[(vergleiche, NN), (frankfurter, NN), (allgemeine, NN), (sonntagszeitung, NN), (12, CD), (august, NN), (2012, CD)]","[vergleiche, frankfurter, allgemeine, sonntagszeitung, 12, august, 2012]"
22,"Nehmen Sie bitte Platz. Die Sitzung ist eröffnet. Guten Morgen, liebe Kolleginnen und Kollegen! Ich begrüße Sie alle herzlich und fange dabei mit den Schriftführerinnen an.",Norbert Lammert,"Dr. Dagmar Enkelmann [DIE LINKE]: Guten Morgen, Herr Präsident!","[dr, dagmar, enkelmann, linke, guten, morgen, herr, präsident]","[(dr, NN), (dagmar, NN), (enkelmann, NN), (linke, NN), (guten, VBP), (morgen, NN), (herr, NN), (präsident, NN)]","[dr, dagmar, enkelmann, linke, guten, morgen, herr, präsident]"
23,Das war unsere Kollegin Ewa Klamt für die Fraktion der CDU/CSU. Wir sind am Ende unserer Aussprache zu diesem Tagesordnungspunkt. Wir kommen nun zur Abstimmung über die Beschlussempfehlung des...,Eduard Oswald,f,[f],"[(f, NN)]",[f]
24,Diese Begleiterklärung nehmen wir zu Protokoll.1,Hermann Otto Solms,,[],[],[]
25,Das Wort hat nun Priska Hinz für die Fraktion Bünd nis 90/Die Grünen. Priska Hinz,Wolfgang Thierse,Herborn,[herborn],"[(herborn, NN)]",[herborn]
29,"Darf ich einmal um ein Signal bitten, inwieweit alle Kolleginnen und Kollegen die Möglichkeit hatten, an der Abstimmung teilzunehmen? – Mir wird signalisiert, dass sich keine Kolleginnen und Kol...",Petra Pau,Lachen bei der SPD – Sören Bartol [SPD]: Der sitzt doch da!,"[lachen, spd, sören, bartol, spd, sitzt]","[(lachen, NN), (spd, NN), (sören, NN), (bartol, NN), (spd, NN), (sitzt, NN)]","[lachen, spd, sören, bartol, spd, sitzt]"
31,"Guten Morgen, liebe Kolleginnen und Kollegen! Die Sitzung ist eröffnet. Ich rufe den Tagesordnungspunkt 43 auf: Erste Beratung des von der Bundesregierung eingebrachten Entwurfs eines Gesetzes...",Katrin Göring-Eckardt,f,[f],"[(f, NN)]",[f]


In [ ]:
#split the components and write into new dataframe
data = pd.DataFrame()
Redebeitrag = []
Person = []
Unterbrechung = []

for x in Sub:
    string = re.split(':', str(x), 1)
    string1 = re.split('\\(|\\)', str(string[1]))
    Person.append(string[0])
    Redebeitrag.append(string1[0])
    Unterbrechung.append(string1[1])

data["Redeinhalt"] = Redebeitrag
data["Vortragende_Person"] = Person
data["Unterbrechung"] = Unterbrechung

In [201]:
#split the interruption-components and add to dataframe
#funktioniert nicht da die rows nicht alle so aufgebaut sind - if schleife einbauen mit if row == regex (Volker Kauder [CDU/CSU]: Jetzt aber!]
pers_unter = []
rede_unter = []
partei_unter = []

for row in data["Unterbrechung"]:
    if re.findall(r':', row):
        string = re.split(':', str(row))
        str_pers = re.split("\s\[\w*?.\w*\]$", string[0])
        str_part = re.search("\s\[\w*?.\w*\]$", string[0])
        str_part = re.sub("\[?\]?", "", str_part[0]) # delete []
        pers_unter.append(str_pers[0])
        rede_unter.append(string[1])
        partei_unter.append(str_part)
    else:
        pers_unter.append("0")
        partei_unter.append("0")
        rede_unter.append(row)

data["Person_Unterbrechung"] = pers_unter
data["Partei_Unterbrechung"] = partei_unter
data["Redeinhalt_Unterbrechung"] = rede_unter

['Dr. Heinrich L. Kolb [FDP]', ' Gewagte These!'] ['Lachen bei der SPD – Sören Bartol', '']
['Dr. Dagmar Enkelmann [DIE LINKE]', ' Guten Morgen, Herr Präsident!'] ['Lachen bei der SPD – Sören Bartol', '']
['Lachen bei der SPD – Sören Bartol [SPD]', ' Der sitzt doch da!'] ['Lachen bei der SPD – Sören Bartol', '']


In [202]:
data

,Redeinhalt,Vortragende_Person,Unterbrechung,TOKENS,POS,LEMMATA,Redeinhalt_Unterbrechung,Person_Unterbrechung,Partei_Unterbrechung
1,"Die Sitzung ist eröffnet. Guten Morgen, liebe Kolleginnen und Kollegen! Wir setzen unsere Haushaltsberatungen – Tagesordnungspunkt 2 – fort: Erste Beratung des von der Bundesregierung eingebrac...",Norbert Lammert,Haushaltsgesetz 2010,"[haushaltsgesetz, 2010]","[(haushaltsgesetz, NN), (2010, CD)]","[haushaltsgesetz, 2010]",Haushaltsgesetz 2010,0,0
13,Ich eröffne die unterbrochene Sitzung wieder und teile die Ergebnisse der drei namentlichen Abstimmungen mit. Zunächst gebe ich das von den Schriftführerinnen und Schriftführern ermittelte Erge...,Wolfgang Thierse,D,[d],"[(d, NN)]",[d],D,0,0
14,"„Die Deutschen haben ein Recht auf Widerstand gegen die Überfremdung“, Ausdrücke auf Plakaten wie auf denen der CDU: „Asylmissbrauch beenden“, usw. waren damals auch die Stichworte für die Brandl...",Norbert Geis,Dr. Heinrich L. Kolb [FDP]: Gewagte These!,"[dr, heinrich, l, kolb, fdp, gewagte, these]","[(dr, NN), (heinrich, NN), (l, NN), (kolb, NN), (fdp, NN), (gewagte, NN), (these, DT)]","[dr, heinrich, l, kolb, fdp, gewagte, these]",Gewagte These!,Dr. Heinrich L. Kolb,FDP
16,"Unterstützt die Bundesregierung finanziell die von Indo nesien bestätigte Lieferung von 130 gebrauchten Leopard-2Panzern, und sind diese Panzer – ebenso wie die für SaudiArabien vorgesehene Version",Rolf Mützenich,vergleiche Frankfurter Allgemeine Sonntagszeitung vom 12. August 2012,"[vergleiche, frankfurter, allgemeine, sonntagszeitung, 12, august, 2012]","[(vergleiche, NN), (frankfurter, NN), (allgemeine, NN), (sonntagszeitung, NN), (12, CD), (august, NN), (2012, CD)]","[vergleiche, frankfurter, allgemeine, sonntagszeitung, 12, august, 2012]",vergleiche Frankfurter Allgemeine Sonntagszeitung vom 12. August 2012,0,0
22,"Nehmen Sie bitte Platz. Die Sitzung ist eröffnet. Guten Morgen, liebe Kolleginnen und Kollegen! Ich begrüße Sie alle herzlich und fange dabei mit den Schriftführerinnen an.",Norbert Lammert,"Dr. Dagmar Enkelmann [DIE LINKE]: Guten Morgen, Herr Präsident!","[dr, dagmar, enkelmann, linke, guten, morgen, herr, präsident]","[(dr, NN), (dagmar, NN), (enkelmann, NN), (linke, NN), (guten, VBP), (morgen, NN), (herr, NN), (präsident, NN)]","[dr, dagmar, enkelmann, linke, guten, morgen, herr, präsident]","Guten Morgen, Herr Präsident!",Dr. Dagmar Enkelmann,DIE LINKE
23,Das war unsere Kollegin Ewa Klamt für die Fraktion der CDU/CSU. Wir sind am Ende unserer Aussprache zu diesem Tagesordnungspunkt. Wir kommen nun zur Abstimmung über die Beschlussempfehlung des...,Eduard Oswald,f,[f],"[(f, NN)]",[f],f,0,0
24,Diese Begleiterklärung nehmen wir zu Protokoll.1,Hermann Otto Solms,,[],[],[],,0,0
25,Das Wort hat nun Priska Hinz für die Fraktion Bünd nis 90/Die Grünen. Priska Hinz,Wolfgang Thierse,Herborn,[herborn],"[(herborn, NN)]",[herborn],Herborn,0,0
29,"Darf ich einmal um ein Signal bitten, inwieweit alle Kolleginnen und Kollegen die Möglichkeit hatten, an der Abstimmung teilzunehmen? – Mir wird signalisiert, dass sich keine Kolleginnen und Kol...",Petra Pau,Lachen bei der SPD – Sören Bartol [SPD]: Der sitzt doch da!,"[lachen, spd, sören, bartol, spd, sitzt]","[(lachen, NN), (spd, NN), (sören, NN), (bartol, NN), (spd, NN), (sitzt, NN)]","[lachen, spd, sören, bartol, spd, sitzt]",Der sitzt doch da!,Lachen bei der SPD – Sören Bartol,SPD
31,"Guten Morgen, liebe Kolleginnen und Kollegen! Die Sitzung ist eröffnet. Ich rufe den Tagesordnungspunkt 43 auf: Erste Beratung des von der Bundesregierung eingebrachten Entwurfs eines Gesetzes...",Katrin Göring-Eckardt,f,[f],"[(f, NN)]",[f],f,0,0


In [203]:
#split the interruption-components and add to dataframe
#funktioniert nicht da die rows nicht alle so aufgebaut sind - if schleife einbauen mit if row == regex (Volker Kauder [CDU/CSU]: Jetzt aber!]
'''pers_unter = []
rede_unter = []
partei_unter = []

for row in data["Unterbrechung"]:
    string = re.split('[|]:', str(row))
    pers_unter.append(string[0])
    partei_unter.append(string[1])
    rede_unter.append(string[2])

data["Redeinhalt_Unterbrechung"] = rede_unter
data["Person_Unterbrechung"] = pers_unter
data["Partei_Unterbrechung"] = partei_unter'''

'pers_unter = []\nrede_unter = []\npartei_unter = []\n\nfor row in data["Unterbrechung"]:\n    string = re.split(\'[|]:\', str(row))\n    pers_unter.append(string[0])\n    partei_unter.append(string[1])\n    rede_unter.append(string[2])\n\ndata["Redeinhalt_Unterbrechung"] = rede_unter\ndata["Person_Unterbrechung"] = pers_unter\ndata["Partei_Unterbrechung"] = partei_unter'

# 3. Datenverarbeitung

In [18]:
#case conversion & tokenizer
tokenizer = RegexpTokenizer(r'\w+')
data["TOKENS"] = data.apply (lambda row: tokenizer.tokenize(str(row["Unterbrechung"].lower())), axis=1)

In [19]:
#stopwords removal
def remove_stopwords_from_list(list_in, language):
    return [a for a in list_in if a not in nltk.corpus.stopwords.words(language)]

In [20]:
data["TOKENS"] = data.apply(lambda row: remove_stopwords_from_list(row["TOKENS"], "german"), axis=1)

In [21]:
'''##doesn`t work
nlp = spacy.load("de_core_news_md")
nlp.max_length = 2000000

words = {}
for row in df["TOKENS"]:
    doc = nlp(str(row))
    for token in doc.ents:
        words = {token.text: token.label_}
        
nlp = spacy.load("de_core_news_md")
nlp.max_length = 2000000
liste = []
doc = nlp(str(df["TEXT_SPLIT"]))
liste.append([(X.text, X.label_) for X in doc.ents])
df["NER"] = liste'''

'##doesn`t work\nnlp = spacy.load("de_core_news_md")\nnlp.max_length = 2000000\n\nwords = {}\nfor row in df["TOKENS"]:\n    doc = nlp(str(row))\n    for token in doc.ents:\n        words = {token.text: token.label_}\n        \nnlp = spacy.load("de_core_news_md")\nnlp.max_length = 2000000\nliste = []\ndoc = nlp(str(df["TEXT_SPLIT"]))\nliste.append([(X.text, X.label_) for X in doc.ents])\ndf["NER"] = liste'

In [ ]:
'''#doesn't work
import nltk

from nltk import ne_chunk, pos_tag, word_tokenize
from nltk.tree import Tree
nltk.download("punkt")
nltk.download('maxent_ne_chunker')
nltk.download('words')

text =
Alterspräsident Dr. Heinz Riesenhuber
und es wird weltweit geteilt. Die weltweite Gemein-
schaft, die Wissensgesellschaft bietet jedem offenen Zu-
griff auf alles, wenn er alles, was er hat, mit einbringt. Es
ist eine Gesellschaft, die aus Wissen lebt, die mit Wissen
wächst, die mit Wissen verantwortlich umzugehen ver-
steht, in der Wissen überall und jederzeit für jeden zu-
gänglich ist.

Es ist die Welt der Quanten, die wir langsam verste-
hen lernen  von der Vision des Quantencomputers bis
zur Nanotechnologie, wo im Allerkleinsten die Materie
andere Eigenschaften hat. Es ist die Welt der Gene, die
neue Möglichkeiten schafft. Nur ein Viertel der Krank-
heiten, die wir kennen, können wir an der Wurzel fassen.
Gentechnologie eröffnet die Chance, dass wir Krankhei-
ten heilen, dass wir menschliches Leid lindern, dass wir
gleichzeitig aus Wissen neue Märkte, neue Arbeitsplätze
und wirtschaftliches Wachstum schaffen.

Es ist die Welt der Computer, die schon jetzt unseren
Alltag beherrscht. Die Leistung der Chips verdoppelt
sich alle 18 Monate, und es halbiert sich ihr Preis. Da-
raus entstehen Internet und Unterhaltungselektronik.
Daraus entsteht eine Fülle von Möglichkeiten in Chemie
und im Maschinenbau. Daraus entsteht eine Fülle von
Möglichkeiten im Automobilbau. Daraus entstehen lau-
ter Arbeitsplätze, die für unsere Zukunft wichtig sind, in
einem Bereich, wo Deutschland immer stark war.
Deutschland war selten in den Spitzentechnologien
vorne, aber als Meister der Systeme wurden aus den ein-
zelnen Techniken neue Produkte geschaffen.

Der Chip ist Silizium, und Silizium ist Sand  und
Sand ist reichlich vorhanden. Und dazu braucht man
noch Intelligenz. Intelligenz, so sagt man uns, ist belie-
big vorhanden. In der Praxis findet man gelegentlich
Knappheiten.

(Heiterkeit bei Abgeordneten der CDU/CSU
und der FDP)

Aber sie wächst nach; es bleibt Hoffnung. Insofern ha-
ben wir hier die Chance eines Wachstums des guten Ge-
wissens, das die Wirklichkeit ändert und Zukunft für alle
schafft.

Dass diese Welt gelingt, hängt davon ab, ob wir die
Menschen haben, die sie gestalten, die Freude daran ha-
ben und die ihre Chance darin sehen. Wir werden uns
über vieles streiten, und wir haben uns über vieles ge-
stritten  manchmal mit intensiver Herzlichkeit. Aber
wir sind uns doch wohl weitestgehend darin einig, dass
Bildung und Forschung in diesen Jahren hohe Priorität
haben. Dafür braucht es nicht allein Geld  das auch ,
sondern auch Konzepte und unsere Bereitschaft, denen,
die hier gestalten, die Freude daran nicht zu nehmen.
Wir haben in diesen Jahren in einer Welt im Wandel die
Schulen umzubauen und aufzubauen. Wir haben gute
Schulen, und seit PISA sind sie noch besser geworden;
aber wir wissen, dass wir noch mehr erreichen können.

In der Föderalismusreform haben wir den Ländern die
Zuständigkeiten für die Bildung weitgehend zugeschrie-
ben. Sie wetteifern um das beste Schulsystem und um
die besten Chancen. Den Universitäten haben wir durch
einen Wettbewerb um Exzellenz, der über zehn Jahre an-
gelegt worden ist, den Ehrgeiz vermittelt, die Besten zu
sein und von jedem anderen zu lernen. Der Forschung
haben wir langfristig steigende Mittel zugesagt.

Was wir hier machen, ist eine Investition dafür, dass
wir stärker sind, wenn wir aus der Krise herauskommen.
Wir brauchen diese Stärke. Es gibt Aufgaben über das
Tagesgeschäft hinaus, die langfristig sind und in unter-
schiedlichen Bereichen angegangen werden müssen. Die
Menschen werden älter. Das ist ein Erfolg der Medizin.
Das ist ein Erfolg der Arbeitsbedingungen, die besser
geworden sind. Das ist ein Erfolg der Sozialsysteme.
Das ist also eine erfreuliche Sache und kein Problem.
Das Problem liegt darin, dass wir nicht rechtzeitig so
viele Kinder gezeugt haben, wie wir es hätten tun sollen.

(Heiterkeit)

Was hier und jetzt anzugehen ist,

(Renate Künast [BÜNDNIS 90/DIE GRÜ-
NEN]: Das wollen Sie jetzt angehen?  Heiter-
keit)

ist, dass wir in dieser Gesellschaft, die altert, die Kreati-
vität bewahren und das dritte Lebensalter nicht als ein
Zu-Ende-Leben des Lebens verstehen, sondern als neuen
Raum der Gestaltung mit einer Freiheit, die man über
das ganze frühere Leben nicht hatte. Man kann wählen;
das gilt sowohl für die Freizeit, das Ehrenamt und, wenn
man es will und kann, die Arbeit. Dies so zu organisie-
ren, dass die Menschen diese Chance ergreifen, dass sie
mitgestalten und aktiv dabei sind, ist eine unserer großen
Aufgaben.


nltk_results = ne_chunk(pos_tag(word_tokenize(text)))
for nltk_result in nltk_results:
    if type(nltk_result) == Tree:
        name = ''
        for nltk_result_leaf in nltk_result.leaves():
            name += nltk_result_leaf[0] + ' '
        print ('Type: ', nltk_result.label(), 'Name: ', name)'''

In [22]:
#pos tagging
words = []
for row in data["TOKENS"]:
    tagged_tokens = nltk.pos_tag(row)
    words_ = words.append(tagged_tokens)

data["POS"] = words

In [23]:
#lemmatizer
lemmatizer = WordNetLemmatizer()
data["LEMMATA"] = data.apply(lambda row: [lemmatizer.lemmatize(word) for word in row["TOKENS"]], axis=1)

In [24]:
data

,Redeinhalt,Vortragende_Person,Unterbrechung,TOKENS,POS,LEMMATA
1,"Die Sitzung ist eröffnet. Guten Morgen, liebe Kolleginnen und Kollegen! Wir setzen unsere Haushaltsberatungen – Tagesordnungspunkt 2 – fort: Erste Beratung des von der Bundesregierung eingebrac...",Norbert Lammert,Haushaltsgesetz 2010,"[haushaltsgesetz, 2010]","[(haushaltsgesetz, NN), (2010, CD)]","[haushaltsgesetz, 2010]"
13,Ich eröffne die unterbrochene Sitzung wieder und teile die Ergebnisse der drei namentlichen Abstimmungen mit. Zunächst gebe ich das von den Schriftführerinnen und Schriftführern ermittelte Erge...,Wolfgang Thierse,D,[d],"[(d, NN)]",[d]
14,"„Die Deutschen haben ein Recht auf Widerstand gegen die Überfremdung“, Ausdrücke auf Plakaten wie auf denen der CDU: „Asylmissbrauch beenden“, usw. waren damals auch die Stichworte für die Brandl...",Norbert Geis,Dr. Heinrich L. Kolb [FDP]: Gewagte These!,"[dr, heinrich, l, kolb, fdp, gewagte, these]","[(dr, NN), (heinrich, NN), (l, NN), (kolb, NN), (fdp, NN), (gewagte, NN), (these, DT)]","[dr, heinrich, l, kolb, fdp, gewagte, these]"
16,"Unterstützt die Bundesregierung finanziell die von Indo nesien bestätigte Lieferung von 130 gebrauchten Leopard-2Panzern, und sind diese Panzer – ebenso wie die für SaudiArabien vorgesehene Version",Rolf Mützenich,vergleiche Frankfurter Allgemeine Sonntagszeitung vom 12. August 2012,"[vergleiche, frankfurter, allgemeine, sonntagszeitung, 12, august, 2012]","[(vergleiche, NN), (frankfurter, NN), (allgemeine, NN), (sonntagszeitung, NN), (12, CD), (august, NN), (2012, CD)]","[vergleiche, frankfurter, allgemeine, sonntagszeitung, 12, august, 2012]"
22,"Nehmen Sie bitte Platz. Die Sitzung ist eröffnet. Guten Morgen, liebe Kolleginnen und Kollegen! Ich begrüße Sie alle herzlich und fange dabei mit den Schriftführerinnen an.",Norbert Lammert,"Dr. Dagmar Enkelmann [DIE LINKE]: Guten Morgen, Herr Präsident!","[dr, dagmar, enkelmann, linke, guten, morgen, herr, präsident]","[(dr, NN), (dagmar, NN), (enkelmann, NN), (linke, NN), (guten, VBP), (morgen, NN), (herr, NN), (präsident, NN)]","[dr, dagmar, enkelmann, linke, guten, morgen, herr, präsident]"
23,Das war unsere Kollegin Ewa Klamt für die Fraktion der CDU/CSU. Wir sind am Ende unserer Aussprache zu diesem Tagesordnungspunkt. Wir kommen nun zur Abstimmung über die Beschlussempfehlung des...,Eduard Oswald,f,[f],"[(f, NN)]",[f]
24,Diese Begleiterklärung nehmen wir zu Protokoll.1,Hermann Otto Solms,,[],[],[]
25,Das Wort hat nun Priska Hinz für die Fraktion Bünd nis 90/Die Grünen. Priska Hinz,Wolfgang Thierse,Herborn,[herborn],"[(herborn, NN)]",[herborn]
29,"Darf ich einmal um ein Signal bitten, inwieweit alle Kolleginnen und Kollegen die Möglichkeit hatten, an der Abstimmung teilzunehmen? – Mir wird signalisiert, dass sich keine Kolleginnen und Kol...",Petra Pau,Lachen bei der SPD – Sören Bartol [SPD]: Der sitzt doch da!,"[lachen, spd, sören, bartol, spd, sitzt]","[(lachen, NN), (spd, NN), (sören, NN), (bartol, NN), (spd, NN), (sitzt, NN)]","[lachen, spd, sören, bartol, spd, sitzt]"
31,"Guten Morgen, liebe Kolleginnen und Kollegen! Die Sitzung ist eröffnet. Ich rufe den Tagesordnungspunkt 43 auf: Erste Beratung des von der Bundesregierung eingebrachten Entwurfs eines Gesetzes...",Katrin Göring-Eckardt,f,[f],"[(f, NN)]",[f]


In [ ]:
'''#create new Dataframe and copy some old columns
data = df[["NR","DATUM","TEXT_SPLIT"]].copy()'''